______

# Welcome to Quantum Blackjack
### A Game of Superposition, Entanglement, and Strategy

This game introduces a unique twist on the classic game of Blackjack, infusing it with the fascinating principles of quantum mechanics. While the objective remains familiar—to get a hand value as close to **17** as possible without going over—the way you play is fundamentally different. Instead of a standard deck, you are dealt a combination of classical and "quantum" cards, transforming the game from one of simple chance into a strategic exercise in probability management.

---

## The Quantum Cards

At the heart of the game lies the concept of the quantum card. Unlike a classical card, which has a single, definite value, a quantum card exists in a state of **superposition**. 

This means it represents multiple possible values (from `1` to `8`) *at the same time*, each with a specific probability of being the final outcome. You can think of it not as a card with a hidden value, but as a cloud of potential values that will only collapse into a single, concrete number when a **measurement** occurs.

This game visualizes these odds for you, showing the initial probability distribution for each of your quantum cards.

## Key Quantum Principles in Your Hand

This game leverages three core quantum phenomena to create a unique strategic landscape:

* **Superposition and Probabilistic Strategy:** The initial state of your quantum cards is set by Hadamard (`H`) and rotation (`RY`) gates. This gives you a starting set of probabilities. Your key action, **`[H] Re-Shuffle`**, allows you to apply another Hadamard gate to one of your quantum cards, directly altering its probability distribution and allowing you to strategically manipulate your odds before committing to a measurement.

* **Entanglement - A Shared Fate:** The most intriguing feature of this game is **entanglement**. The quantum cards of Player 1 and Player 2 are linked using CNOT (`CX`) gates. This creates a deep, non-classical connection between them. The measurement outcome of one player's card will instantly influence the outcome of the other's. As the code's design implies, their results are correlated, meaning you aren't just playing against the dealer; you are in a quantum duel with the other player.

* **Measurement - The Collapse of Possibility:** The ultimate decision in the game is to **`[M]easure`**. This action forces the quantum system to collapse. At that moment, every quantum card in play—for you, the other player, and the dealer—simultaneously settles into a single, definite value. The timing of this measurement is the game's central strategic pivot. Do you measure early with a 2-card hand, or do you let the quantum state evolve, aiming for a 3-card hand but giving your opponent a chance to alter the entangled system?

---

In essence, this Quantum Blackjack challenges you to think beyond fixed odds. You must weigh probabilities, strategically modify them, and consider how your actions will ripple through the entangled state to affect your opponent, all before making the decisive choice to collapse the quantum wave function and reveal your final hand.

In [8]:
import asyncio
import random
from typing import List, Dict, Tuple
import ipywidgets as widgets
from ipywidgets import GridspecLayout
import numpy as np
from IPython.display import display, clear_output
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
import nest_asyncio

# Apply the patch to allow asyncio to work correctly in Jupyter
nest_asyncio.apply()

# --- Game Constants ---
TARGET_SCORE = 17
DEALER_STANDS_THRESHOLD = 14
MIN_CARD_VALUE = 1
MAX_CARD_VALUE = 8
NUM_QUANTUM_CARDS_PER_PLAYER = 2

class QuantumCardGameGUI:
    """
    GUI version with two independent, reverse-entangled pairs for players,
    using H+RY gates to create non-uniform initial card distributions.
    """

    def __init__(self):
        """Initializes the game, simulator, and all GUI components."""
        self.backend = AerSimulator()
        self.qc: QuantumCircuit = None
        self.player_scores: Dict[str, int] = {}
        self.initial_cards: Dict[str, int] = {}
        self.player_hands: Dict[str, List[int]] = {}
        self.player_turn_order: List[int] = []
        self.current_player_index = 0
        self.game_over = False
        self.current_round = 1
        self.measure_level = 3
        self._setup_gui_elements()

    def _get_card_image(self, card_value: str or int, revealed=True) -> widgets.Image:
        image_name = str(card_value) if revealed else 'back'
        try:
            # Assumes you have a 'card_images' folder in the same directory
            with open(f"card_images/{image_name}.png", "rb") as f:
                return widgets.Image(value=f.read(), format='png', width=80, height=120)
        except FileNotFoundError:
            # Fallback if images are not found
            return widgets.Label(f" {card_value} ")

    def _setup_gui_elements(self):
        """Creates and organizes all the visual components of the game."""
        self.measure_button = widgets.Button(description="Measure", button_style='danger', layout=widgets.Layout(width='auto', height='auto'))
        self.pass_button = widgets.Button(description="Pass", button_style='info', layout=widgets.Layout(width='auto', height='auto'))
        self.reshuffle_button = widgets.Button(description="Re-Shuffle", button_style='success', layout=widgets.Layout(width='auto', height='auto'))
        self.card_choice_radio = widgets.RadioButtons(options=['1', '2'], description='', layout=widgets.Layout(width='auto'))
        
        self.action_box = GridspecLayout(2, 3, grid_gap='5px', layout=widgets.Layout(width='90%', margin='10px 0 0 0'))
        self.action_box[0, 0] = self.measure_button
        self.action_box[0, 1] = self.pass_button
        self.action_box[0, 2] = self.reshuffle_button
        q_card_label = widgets.Label("Q-Card:")
        radio_with_label = widgets.HBox([q_card_label, self.card_choice_radio])
        self.action_box[1, 2] = radio_with_label

        self.dealer_cards_box = widgets.HBox([])
        self.player1_cards_box = widgets.HBox([])
        self.player2_cards_box = widgets.HBox([])
        self.dealer_score_label = widgets.Label("Dealer's Hand")
        self.player1_score_label = widgets.Label("Player 1's Hand")
        self.player2_score_label = widgets.Label("Player 2's Hand")

        player_box_layout = widgets.Layout(border='2px solid #DDDDDD', padding='10px', align_items='center', width='400px', margin='5px')
        dealer_box_layout = widgets.Layout(border='2px solid #555555', padding='10px', align_items='center', width='400px', margin='5px 0 20px 0')
        self.dealer_box = widgets.VBox([self.dealer_score_label, self.dealer_cards_box], layout=dealer_box_layout)
        self.player1_box = widgets.VBox([self.player1_score_label, self.player1_cards_box], layout=player_box_layout)
        self.player2_box = widgets.VBox([self.player2_score_label, self.player2_cards_box], layout=player_box_layout)
        
        self.message_label = widgets.HTML(value="<p style='color:black;'>Welcome! Press 'Start Game' to begin.</p>")
        self.round_label = widgets.HTML(value="")
        self.hand_details_label = widgets.HTML(value="") 
        self.start_button = widgets.Button(description="Start Game", button_style='primary')
        self.start_button.on_click(lambda b: asyncio.create_task(self.start_game(b)))
        
        players_row = widgets.HBox([self.player1_box, self.player2_box], layout=widgets.Layout(justify_content='center'))
        self.game_area = widgets.VBox([self.dealer_box, players_row], layout=widgets.Layout(align_items='center'))

        self.header = widgets.HTML("<h1>Quantum Black Jack</h1>")
        self.footer = widgets.VBox([self.round_label, self.message_label, self.hand_details_label, self.start_button])
        self.app_layout = widgets.AppLayout(header=self.header, center=self.game_area, footer=self.footer, pane_widths=['0', '100%', '0'], grid_gap="10px")

    def _set_message(self, msg, color="black"):
        self.message_label.value = f"<p style='color:{color};'>{msg}</p>"

    def _set_round_message(self, msg, color="black"):
        self.round_label.value = f"<p style='color:{color}; font-style:italic;'>{msg}</p>"

    def _generate_random_card(self) -> int:
        return np.random.randint(MIN_CARD_VALUE, MAX_CARD_VALUE + 1)

    def _get_card_value_from_binary(self, binary_string: str) -> int:
        return int(binary_string, 2) + 1


    def _prepare_player_card_with_rotations(self, qubits: List) -> List[float]:
        """
        Applies H and RY gates to a set of qubits and returns the
        resulting probability distribution.
        """
        # Apply gates to the main game circuit
        self.qc.h(qubits)
        rotation_angles = []
        for qubit in qubits:
            angle = np.random.uniform(0, 2 * np.pi)
            self.qc.ry(angle, qubit)
            rotation_angles.append(angle)

        # Use a temporary circuit to calculate the resulting probabilities
        temp_qc = QuantumCircuit(3)
        temp_qc.h(range(3))
        for i, angle in enumerate(rotation_angles):
            temp_qc.ry(angle, i)

        # Calculate statevector and probabilities
        state = Statevector.from_instruction(temp_qc)
        return state.probabilities()

    def _prepare_quantum_circuit(self):
        """
        MODIFIED: Prepares the circuit using H+RY gates for players and displays
        the resulting probability distribution in the GUI.
        """
        num_qubits = NUM_QUANTUM_CARDS_PER_PLAYER * 3
        self.dealer_q = QuantumRegister(num_qubits, 'dealer')
        self.player1_q = QuantumRegister(num_qubits, 'player1')
        self.player2_q = QuantumRegister(num_qubits, 'player2')
        self.dealer_c = ClassicalRegister(num_qubits, 'dealer_res')
        self.player1_c = ClassicalRegister(num_qubits, 'player1_res')
        self.player2_c = ClassicalRegister(num_qubits, 'player2_res')

        self.qc = QuantumCircuit(self.dealer_q, self.player1_q, self.player2_q, self.dealer_c, self.player1_c, self.player2_c)
        self.qc.barrier()
        
        # 1. Dealer's cards are independent (simple superposition)
        self.qc.h(self.dealer_q)
        self.qc.barrier()
        
        # 2. Player cards are prepared with H and RY gates
        # Card 1 (qubits 3,4,5)
        probs_card1 = self._prepare_player_card_with_rotations(self.player1_q[3:6])
        self.qc.barrier()
        # Card 2 (qubits 0,1,2)
        probs_card2 = self._prepare_player_card_with_rotations(self.player1_q[0:3])
        self.qc.barrier()

        # Update GUI with the probability distributions
        dist_html = "<h4 style='color:black; margin-bottom:5px;'>Initial Player Card Probabilities</h4>"
        dist_html += "<div style='display: flex; justify-content: space-around;'>"
        dist_html += "<div style='width: 45%;'><b>Quantum Card 1:</b><ul style='margin-top:0;'>"
        for i, p in enumerate(probs_card1):
            dist_html += f"<li>Value {i+1}: {p:.1%}</li>"
        dist_html += "</ul></div>"
        dist_html += "<div style='width: 45%;'><b>Quantum Card 2:</b><ul style='margin-top:0;'>"
        for i, p in enumerate(probs_card2):
            dist_html += f"<li>Value {i+1}: {p:.1%}</li>"
        dist_html += "</ul></div></div>"
        self.hand_details_label.value = dist_html

        # 3. Create the first entangled pair (Card 1)
        self.qc.cx(self.player1_q[3], self.player2_q[5])
        self.qc.cx(self.player1_q[4], self.player2_q[4])
        self.qc.cx(self.player1_q[5], self.player2_q[3])
        self.qc.barrier()

        # 4. Create the second, independent entangled pair (Card 2)
        self.qc.cx(self.player1_q[0], self.player2_q[2])
        self.qc.cx(self.player1_q[1], self.player2_q[1])
        self.qc.cx(self.player1_q[2], self.player2_q[0])
        self.qc.barrier()


    async def _determine_turn_order(self):
        self._set_message("Flipping a quantum coin to determine turn order...")
        await asyncio.sleep(1.5)
        coin_qc = QuantumCircuit(1, 1); coin_qc.h(0); coin_qc.measure(0, 0)
        result = self.backend.run(coin_qc, shots=1, memory=True).result().get_memory()[0]
        if result == '0':
            self._set_message("Coin shows |0>. Player 1 will play first this round.")
            return [1, 2]
        else:
            self._set_message("Coin shows |1>. Player 2 will play first this round.")
            return [2, 1]

    def _add_measurements_to_circuit(self):
        self._set_round_message("Locking in final results... Collapsing quantum states...")
        if self.measure_level == 2:
            self._set_message("Measuring first quantum cards only...")
            self.qc.measure(self.dealer_q[3:6], self.dealer_c[3:6]); self.qc.measure(self.player1_q[3:6], self.player1_c[3:6]); self.qc.measure(self.player2_q[3:6], self.player2_c[3:6])
        else:
            self._set_message("Measuring all quantum cards...")
            self.qc.measure(self.dealer_q, self.dealer_c); self.qc.measure(self.player1_q, self.player1_c); self.qc.measure(self.player2_q, self.player2_c)
    
    def _update_turn_highlight(self):
        self.player1_box.layout.border = '2px solid #DDDDDD'; self.player2_box.layout.border = '2px solid #DDDDDD'
        self.player1_box.children = [c for c in self.player1_box.children if c is not self.action_box]
        self.player2_box.children = [c for c in self.player2_box.children if c is not self.action_box]
        if self.game_over: return
        player_num = self.player_turn_order[self.current_player_index]
        self._set_message(f"It's Player {player_num}'s turn.")
        if player_num == 1:
            self.player1_box.layout.border = '3px solid #1f77b4'
            self.player1_box.children = list(self.player1_box.children) + [self.action_box]
        else:
            self.player2_box.layout.border = '3px solid #ff7f0e'
            self.player2_box.children = list(self.player2_box.children) + [self.action_box]

    async def _on_action_button_clicked(self, b):
        player_num = self.player_turn_order[self.current_player_index]
        if b.description == 'Measure':
            self.measure_level = self.current_round + 1
            self._set_message(f"Player {player_num} measures! Ending game with {self.measure_level}-card hands.")
            await self._end_game()
        elif b.description == 'Pass':
            self._set_message(f"Player {player_num} passes.")
            await self._next_turn()
        elif b.description == 'Re-Shuffle':
            player_q = self.player1_q if player_num == 1 else self.player2_q
            card_choice = self.card_choice_radio.value
            qubits_to_affect = player_q[3:6] if card_choice == '1' else player_q[0:3]
            self._set_message(f"Player {player_num} re-shuffled quantum card {card_choice}.")
            self.qc.h(qubits_to_affect); self.qc.barrier()
            await self._next_turn()

    async def _next_turn(self):
        self.current_player_index += 1
        if self.current_player_index < len(self.player_turn_order):
            self._update_turn_highlight()
        else:
            self.current_round += 1
            if self.current_round > 2:
                self._set_round_message("All players passed the final round.")
                self.measure_level = 3
                await self._end_game()
            else:
                self._set_round_message("--- Round 2: Final Decision on 3-Card Hand ---")
                self.current_player_index = 0
                self.player_turn_order = await self._determine_turn_order()
                await asyncio.sleep(1); self._update_turn_highlight()

    async def _end_game(self):
        self.game_over = True; self._update_turn_highlight(); await asyncio.sleep(1.5)
        self._add_measurements_to_circuit()
        result_str = self.backend.run(self.qc, shots=1, memory=True).result().get_memory()[0]
        await self._process_quantum_results(result_str)
        await self._dealer_final_turn()
        self._generate_final_summary()

    async def _process_quantum_results(self, result_str: str):
        self.hand_details_label.value = "" # Clear probability distributions
        p2_res, p1_res, dealer_res = result_str.split(' ')
        self.player_hands = { "Player 1": [self.initial_cards["Player 1"]], "Player 2": [self.initial_cards["Player 2"]], "Dealer": [self.initial_cards["Dealer"]] }
        
        p1_q1 = self._get_card_value_from_binary(p1_res[0:3][::-1]); self.player_hands["Player 1"].append(p1_q1)
        p2_q1 = self._get_card_value_from_binary(p2_res[0:3][::-1]); self.player_hands["Player 2"].append(p2_q1)
        d_q1 = self._get_card_value_from_binary(dealer_res[0:3][::-1]); self.player_hands["Dealer"].append(d_q1)
        
        if self.measure_level == 3:
            p1_q2 = self._get_card_value_from_binary(p1_res[3:6][::-1]); self.player_hands["Player 1"].append(p1_q2)
            p2_q2 = self._get_card_value_from_binary(p2_res[3:6][::-1]); self.player_hands["Player 2"].append(p2_q2)
            dealer_score_before_q2 = sum(self.player_hands["Dealer"])
            if dealer_score_before_q2 < DEALER_STANDS_THRESHOLD:
                d_q2 = self._get_card_value_from_binary(dealer_res[3:6][::-1]); self.player_hands["Dealer"].append(d_q2)

        for p in self.player_hands: self.player_scores[p] = sum(self.player_hands[p])
        
        self.player1_cards_box.children = [self._get_card_image(c) for c in self.player_hands["Player 1"]]
        self.player2_cards_box.children = [self._get_card_image(c) for c in self.player_hands["Player 2"]]
        self.dealer_cards_box.children = [self._get_card_image(c) for c in self.player_hands["Dealer"]]
        self.player1_score_label.value = f"Player 1 Score: {self.player_scores['Player 1']}"
        self.player2_score_label.value = f"Player 2 Score: {self.player_scores['Player 2']}"
        self.dealer_score_label.value = f"Dealer Score: {self.player_scores['Dealer']}"
        await asyncio.sleep(2)

    async def _dealer_final_turn(self):
        dealer_total = self.player_scores["Dealer"]
        while dealer_total < DEALER_STANDS_THRESHOLD:
            new_card_val = self._generate_random_card()
            self._set_message(f"Dealer total is {dealer_total} (<{DEALER_STANDS_THRESHOLD}). Dealer MUST draw...")
            await asyncio.sleep(2)
            dealer_total += new_card_val
            self.player_hands["Dealer"].append(new_card_val)
            self.player_scores["Dealer"] = dealer_total
            self.dealer_cards_box.children = [self._get_card_image(c) for c in self.player_hands["Dealer"]]
            self.dealer_score_label.value = f"Dealer Score: {dealer_total}"
            self._set_message(f"Dealer draws a {new_card_val}. New total: {dealer_total}")
            await asyncio.sleep(2)
    
    def _generate_final_summary(self):
        self._set_round_message("--- GAME OUTCOME ---", "darkblue")
        winner_message = ""
        valid_scores = {}
        for player, score in self.player_scores.items():
            if score > TARGET_SCORE: winner_message += f"{player} BUSTED with {score}! "
            else: valid_scores[player] = score
        if not valid_scores: winner_message += "Everyone busted! The house wins."
        else:
            max_score = max(valid_scores.values())
            winners = [p for p, s in valid_scores.items() if s == max_score]
            if len(winners) > 1: winner_message += f"It's a TIE between {', '.join(winners)} with {max_score}!"
            else: winner_message += f"CONGRATULATIONS! {winners[0]} wins with a score of {max_score}!"
        self._set_message(winner_message, "green" if "CONGRATULATIONS" in winner_message else "red")
        
        hand_html = "<h4 style='color:black; margin-top:15px;'>--- Revealed Hands ---</h4>"
        for player in ["Player 1", "Player 2", "Dealer"]:
            cards_str = " + ".join(map(str, self.player_hands[player]))
            hand_html += f"<p style='color:black;'><b>{player}:</b> {cards_str} = {self.player_scores[player]}</p>"
        self.hand_details_label.value = hand_html

        self.start_button.description = "Play Again"
        self.footer.children = (*self.footer.children, self.start_button)

    async def start_game(self, b=None):
        self.footer.children = (self.round_label, self.message_label, self.hand_details_label)
        self.game_over = False; self.current_round = 1; self.current_player_index = 0
        self.hand_details_label.value = ""; self.player_hands = {}
        self.initial_cards["Dealer"] = self._generate_random_card()
        self.initial_cards["Player 1"] = self._generate_random_card()
        self.initial_cards["Player 2"] = self._generate_random_card()
        self.dealer_cards_box.children = [self._get_card_image(self.initial_cards["Dealer"]), self._get_card_image('back', False), self._get_card_image('back', False)]
        self.player1_cards_box.children = [self._get_card_image(self.initial_cards["Player 1"]), self._get_card_image('back', False), self._get_card_image('back', False)]
        self.player2_cards_box.children = [self._get_card_image(self.initial_cards["Player 2"]), self._get_card_image('back', False), self._get_card_image('back', False)]
        self.player1_score_label.value = "Player 1's Hand"; self.player2_score_label.value = "Player 2's Hand"; self.dealer_score_label.value = "Dealer's Hand"
        self._prepare_quantum_circuit()
        self._set_round_message("--- Round 1: Play for 2 or 3 Cards? ---")
        self.player_turn_order = await self._determine_turn_order()
        await asyncio.sleep(1); self._update_turn_highlight()

    def display_app(self):
        """Connects button clicks to their actions and displays the GUI."""
        self.measure_button.on_click(lambda b: asyncio.create_task(self._on_action_button_clicked(b)))
        self.pass_button.on_click(lambda b: asyncio.create_task(self._on_action_button_clicked(b)))
        self.reshuffle_button.on_click(lambda b: asyncio.create_task(self._on_action_button_clicked(b)))
        display(self.app_layout)

In [7]:
game = QuantumCardGameGUI()
game.display_app()

AppLayout(children=(HTML(value='<h1>Quantum Black Jack</h1>', layout=Layout(grid_area='header')), VBox(childre…

_____________________